# Running the Calcite SPARQL adapter using the jaydebeapi library

In [ ]:
!python3 -m pip install --user jaydebeapi

In [1]:
import jaydebeapi
import os
import jpype

In [2]:
# Requires to run "mvn package" on calcite-sparql-core first
jar = "../../core/target/calcite-sparql-core-0.0.1-SNAPSHOT.jar"

# The following lines are not always needed, but most of the times they are because
# the JVM is started without the needed JAR file (see for instance
# https://github.com/baztian/jaydebeapi/issues/85)
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk/"
os.environ["CLASSPATH"] = jar

args = "-Djava.class.path=%s" % jar
jvm_path = jpype.getDefaultJVMPath()
jpype.startJVM(jvm_path, args)

In [ ]:
# Although the Jena driver is not used directly, it is needed to load it first
jaydebeapi.connect(
    "org.apache.jena.jdbc.remote.RemoteEndpointDriver", 
    "jdbc:jena:remote:query=https://dbpedia.org/sparql"
)

In [ ]:
query = """
SELECT w3_label, w3_comment, w3_sameAs, xmlns_name, xmlns_homepage, xmlns_depiction
FROM Company
LIMIT 10
"""

with jaydebeapi.connect(
    "org.apache.calcite.avatica.remote.Driver",
    "jdbc:calcite:model=../java/src/main/resources/modelClassRemote.json",
    driver_args={"lex": "JAVA"},
    jars=jar
) as conn:
    with conn.cursor() as curs:
        curs.execute(query)
        print(curs.fetchone())